In [6]:
from flair.models import SequenceTagger
from flair.data import Sentence

repo_dir = '/Users/marijnkoolen/Code/Huygens/republic-project'
flair_dir = f'{repo_dir}/data/embeddings/flair_embeddings/'

# load the model you trained
model_dir = '../../data/embeddings/flair_embeddings/resources/taggers/ner-tbd-DAT-train_1.0-without_flair-mini_batch_size_8/'
train_size = 1.0
layer_name = 'DAT'
use_flair = False
with_flair = 'with_flair' if use_flair else 'without_flair'

model = {}
for layer_name in ['DAT', 'HOE', 'LOC', 'ORG']:
    model_dir = f'{flair_dir}/resources/taggers/ner-tbd-{layer_name}-train_{train_size}-{with_flair}-mini_batch_size_8'

    model[layer_name] = SequenceTagger.load(f'{model_dir}/final-model.pt')



2023-04-25 14:48:39,326 SequenceTagger predicts: Dictionary with 7 tags: O, S-DAT, B-DAT, E-DAT, I-DAT, <START>, <STOP>
2023-04-25 14:48:47,667 SequenceTagger predicts: Dictionary with 7 tags: O, S-HOE, B-HOE, E-HOE, I-HOE, <START>, <STOP>
2023-04-25 14:48:55,824 SequenceTagger predicts: Dictionary with 7 tags: O, S-LOC, B-LOC, E-LOC, I-LOC, <START>, <STOP>
2023-04-25 14:49:04,211 SequenceTagger predicts: Dictionary with 7 tags: O, S-ORG, B-ORG, E-ORG, I-ORG, <START>, <STOP>


In [14]:
correction = [
    {
        "original": "ONtfangen een Misiive van den Heere Gallieris, haar Hoogh Mogende Minister by den Rycksdagh , geschreven te Regensburgh den een en twintighsten deser loopende maandt, houdende advertentie.",
        "corrected": "ONtfangen een Missive van den Heere Gallieris, haar Hoogh Mogende Minister by den Rycksdagh , geschreven te Regensburgh den een en twintighsten deser loopende maandt, houdende advertentie."
    },
    {
        "original": "de Provincien van Zeelandt, van Vrieslandt, en van Overyssel, gereserveert gelycke vryONtfangen een Misive van den OntBeyt, als die van Gelderlandt.",
        "corrected": "de Provincien van Zeelandt, van Vrieslandt, en van Overyssel, gereserveert gelycke vry ONtfangen een Missive van den Ont- Beyt, als die van Gelderlandt."
    },
    {
        "original": "Van Wassenaer, van Brakel, vanden Born, Boreel, Pelt, van Assendelft, RaadtONtfangen een Mifsive van A. van Heurn, pensionaris van Slingelandt.",
        "corrected": "Van Wassenaer, van Brakel, vanden Born, Boreel, Pelt, van Assendelft, Raadt- ONtfangen een Missive van A. van Heurn, pensionaris van Slingelandt."
    }
]

for sent_pair in correction:
    sent_orig = sent_pair['original']
    # create example sentence
    sentence = Sentence(sent_pair['corrected'])

    # predict tags and print
    for layer_name in model:
        model[layer_name].predict(sentence)

        print(sentence.to_tagged_string(), '\n')
    print('\n\n')

    

Sentence[30]: "ONtfangen een Missive van den Heere Gallieris, haar Hoogh Mogende Minister by den Rycksdagh , geschreven te Regensburgh den een en twintighsten deser loopende maandt, houdende advertentie." → ["een en twintighsten deser loopende maandt"/DAT] 

Sentence[30]: "ONtfangen een Missive van den Heere Gallieris, haar Hoogh Mogende Minister by den Rycksdagh , geschreven te Regensburgh den een en twintighsten deser loopende maandt, houdende advertentie." → ["haar Hoogh Mogende Minister by den Rycksdagh"/HOE] 

Sentence[30]: "ONtfangen een Missive van den Heere Gallieris, haar Hoogh Mogende Minister by den Rycksdagh , geschreven te Regensburgh den een en twintighsten deser loopende maandt, houdende advertentie." → ["Regensburgh"/LOC] 

Sentence[30]: "ONtfangen een Missive van den Heere Gallieris, haar Hoogh Mogende Minister by den Rycksdagh , geschreven te Regensburgh den een en twintighsten deser loopende maandt, houdende advertentie." 




Sentence[29]: "de Provincien van Zeeland

In [57]:
from IPython.display import HTML as html_print

print(html_print("<font color='red'>text</font>"))
print('hoi')

<IPython.core.display.HTML object>
hoi


In [68]:
import os


def get_layer_test_file(layer_name):
    gt_dir = os.path.join(repo_dir, 'ground_truth/entities/tag_de_besluiten')
    layer_gt_dir = os.path.join(gt_dir, f'flair_training_{layer_name}')
    return os.path.join(layer_gt_dir, 'test.txt')


def get_token_tag(line):
    if line == '\n':
        return '', '<S>'
    token, tag = line.strip().split(' ')
    #print(token, tag)
    return token, tag


def get_tokens_tags(test_file):
    with open(test_file, 'rt') as fh:
        return [get_token_tag(line) for line in fh]


def read_test_file(test_file):
    tokens_tags = get_tokens_tags(test_file)
    tokens = [token for token, tag in tokens_tags]
    tokens = [token if token != '' else '\n' for token in tokens]
    text = ' '.join(tokens)
    for resolution in text.split('\n'):
        yield resolution
            
            
def highlight_tagged_text(sentence, color):
    tagged_ranges = [(label.data_point.start_position, label.data_point.end_position) for label in sentence.labels]
    print(tagged_ranges)
    text = sentence.text
    for tagged_range in tagged_ranges[::-1]:
        start, end = tagged_range
        print('START:', start, 'END:', end)
        before, tag, after = text[:start], text[start:end], text[end:]
        print('\tBEFORE:', before)
        print('\tTAG:', tag)
        print('\tAFTER:', after)
        text = f"{before}<font color='{color}'>{tag}</font>{after}"
        print('\tTEXT:', text)
        
    return f"<p>{text}</p>"
        
        


In [69]:
LAYER_COLOR = {
    'DAT': 'blue',
    'HOE': 'red',
    'LOC': 'green',
    'ORG': 'purple'
}



layer_name = 'DAT'

texts = []

layer_test_file = get_layer_test_file(layer_name)
for res in read_test_file(layer_test_file):
    print(res)

    for layer in model:
        sentence = Sentence(res)
        model[layer].predict(sentence)
#         print('\n')
        color = LAYER_COLOR[layer]
        text = highlight_tagged_text(sentence, color)
        texts.append(text)
#         for label in sentence.labels:
#             print(label.shortstring, label.score)
#             print(f'{label.data_point.start_position}-{label.data_point.end_position}\t{label.data_point.text}')
#         print(sentence.to_tagged_string())
    break
    
html_print('\n\n'.join(texts))

ONtfangen een Missive van den Lieutetenant Collonel L L. Deyts s, Commandeur van Viissingen n, geschreven te Yperen den eersten deeser loopende maand d, daar by zaan haar Hoog Mogende kennisse e. geevende dat t, in gevolge het verlof geobtineert van sijn Doorlugtige Hoogheid om sig uit sijn Commandement van Vlissingen te moogen absenteeren na Yperen in sijn Guarnisoen n, als Lieutenant Collonel by het eerste Battaillon van den Lieutenant Generaal de Braauw w, hy te Yperen voornoemd was aangekoomen n, en dat hy by absentie van den Commandeur de Casembroot t, sig aldaar den oudsten in rang bevindende e, het commando van den Major Beek op hem genoomen hadde e. WAAR op geen resolutie is gevallen n. 
[(120, 151)]
START: 120 END: 151
	BEFORE: ONtfangen een Missive van den Lieutetenant Collonel L L. Deyts s, Commandeur van Viissingen n, geschreven te Yperen den 
	TAG: eersten deeser loopende maand d
	AFTER: , daar by zaan haar Hoog Mogende kennisse e. geevende dat t, in gevolge het verlof geo

[]


In [60]:
texts

["<p>ONtfangen een Missive van den Lieutetenant Collonel L L. Deyts s, Commandeur van Viissingen n, geschreven te Yperen den <font color=''>eersten deeser loopende maand d</font>, daar by zaan haar Hoog Mogende kennisse e. geevende dat t, in gevolge het verlof geobtineert van sijn Doorlugtige Hoogheid om sig uit sijn Commandement van Vlissingen te moogen absenteeren na Yperen in sijn Guarnisoen n, als Lieutenant Collonel by het eerste Battaillon van den Lieutenant Generaal de Braauw w, hy te Yperen voornoemd was aangekoomen n, en dat hy by absentie van den Commandeur de Casembroot t, sig aldaar den oudsten in rang bevindende e, het commando van den Major Beek op hem genoomen hadde e. WAAR op geen resolutie is gevallen n.</p>",
 "<p>ONtfangen een Missive van den <font color=''>Lieutetenant Collonel</font> L L. Deyts s, <font color=''>Commandeur van Viissingen n</font>, geschreven te Yperen den eersten deeser loopende maand d, daar by zaan haar Hoog Mogende kennisse e. geevende dat t, in

In [50]:
tagged_ranges = [(121, 150)]
for tr in tagged_ranges[::-1]:
    print(tr)

(121, 150)
